In [1]:
import pandas as pd

# Function to split DataFrame into chunks
def split_dataframe(df: pd.DataFrame, chunk_size: int) -> list[pd.DataFrame]:
    return [df.iloc[i : i + chunk_size] for i in range(0, len(df), chunk_size)]


series = pd.read_pickle("./data/pfw/03_sample_asins.pkl")

if not isinstance(series, pd.Series):
    raise Exception("df is not a pd.Series")

df = pd.DataFrame(
    columns=["asin", "review_complete"],
    data={"asin": series, "review_complete": [None] * len(series)},
)

df = df.sort_index()
df.to_pickle("./data/pfw/04_extract_reviews.pkl")